In [ ]:
class RenderObject:
    def calc_distances(self, x):
        raise NotImplementedError()

    def __add__(self, other):
        s = self
        class LambdaRenderObject(RenderObject):
            def calc_distances(self, x):
                m = np.zeros([2,x.shape[1], x.shape[2]])
                m[0,:,:] = s.calc_distances(x)
                m[1,:,:] = other.calc_distances(x)

                return np.min(m, axis=0)
        return LambdaRenderObject()

In [ ]:
class Sphere(RenderObject):
    def __init__(self, radius, position):
        self.radius = radius
        self.position = np.array(position).reshape([3,1,1])

    def calc_distances(self, x):
        return np.linalg.norm(x - self.position, axis=0) - self.radius


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from typing import Tuple

def normalize(x, axis):
    norm = np.linalg.norm(x,2, axis=axis)
    return x / norm[np.newaxis,:,:]

class Camera:
    def __init__(self, aspact_ratio: float, resolution: Tuple[int, int], z: float, scene: RenderObject):
        """
        :param: aspect_ratio - width/height
        :param: resolution - (width, height)
        """
        self.aspact_ratio = aspact_ratio
        self.scene = scene
        self.resolution = resolution
        self.shape_3 = [3, resolution[0], resolution[1]]
        self.shape_1 = [1, resolution[0], resolution[1]]

        # ([x, y, z], width, height)
        self.position_buffer = np.zeros(self.shape_3)
        self.position_buffer[0,:,:] = np.linspace(-aspact_ratio/2, aspact_ratio/2, num=resolution[0])[:, np.newaxis]
        self.position_buffer[1,:,:] = np.linspace(-.5, .5, num=resolution[1])[:, np.newaxis].T
        
        self.direction_buffer = self.position_buffer.copy()
        self.direction_buffer[2, :, :] = z
        self.direction_buffer = normalize(self.direction_buffer, axis=0)

        self.distance_buffer = np.zeros(self.shape_1)


    def calc_distances(self, position_buffer):
        return self.scene.calc_distances(position_buffer)

    def calc_normals(self) -> np.ndarray:
        normals = np.zeros(self.shape_3)
        offset = 0.01
        offset_vec_x = np.array([offset, 0, 0]).reshape([3,1,1])
        offset_vec_y = np.array([0, offset, 0]).reshape([3,1,1])
        offset_vec_z = np.array([0, 0, offset]).reshape([3,1,1])

        normals[0,:,:] = self.distance_buffer[0,:,:] - self.calc_distances(self.position_buffer + offset_vec_x)
        normals[1,:,:] = self.distance_buffer[0,:,:] - self.calc_distances(self.position_buffer + offset_vec_y)
        normals[2,:,:] = self.distance_buffer[0,:,:] - self.calc_distances(self.position_buffer + offset_vec_z)

        normals = normalize(normals, axis=0)

        return normals

    def run(self, n):
        for i in range(n):
            self.distance_buffer = self.calc_distances(self.position_buffer)[np.newaxis,:,:]
            self.position_buffer += self.direction_buffer * self.distance_buffer

        light_vec = np.array([0.5, 0.3, 1.]).reshape([3,1,1])
        light_vec = normalize(light_vec, axis=0)

        hit_mask = self.distance_buffer[0,:,:] < 0.1

        normals = self.calc_normals()
        # print(normals)
        diffuse = np.sum(normals*light_vec, axis=0)
        diffuse = np.clip(diffuse, 0., 1.)
        diffuse[~hit_mask] = 0.0

        return diffuse

In [ ]:
sphere1 = Sphere(1., [0.0, 0.0, 3.0])
sphere2 = Sphere(1., [0.0, 0.5, 3.0])
f = sphere1 + sphere2
     

n=3
c = Camera(16/9, (160*n,90*n), 1, f)
d = c.run(100)

In [ ]:
plt.imshow(d.T)
plt.colorbar()